# Experience : Surconsommation de Nourriture Hyperpalatable

In [35]:
%load_ext autoreload
%autoreload 2

[autoreload of environments.hyperpalatable_env failed: Traceback (most recent call last):
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/hocine/miniconda3/envs/deepdac/lib/python3.9/site-packages/IPython/extens

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import sys
import os
# Ajouter le dossier src au chemin d'importation
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [37]:
import matplotlib.pyplot as plt
import torch 
import numpy as np

In [38]:
from environments.hyperpalatable_env import HyperpalatableEnvironment
from models.qlearning import QLearning

### Configuration de l'environnement

In [39]:
initial_state = torch.tensor([10])  # État interne initial
setpoints = torch.tensor([20])  # Setpoint pour l'état optimal
weights = torch.tensor([1.0])  # Poids de l'état
exponents = [2, 2]  # Non-linéarité du drive
effects = {0: torch.tensor([1]), 1: torch.tensor([2])}  # Effets des actions
palatability_bonus = 5  # Bonus pour nourriture hyperpalatable
normal_food_reward = 2  # Récompense pour nourriture normale

# Instanciation de l'environnement
env = HyperpalatableEnvironment(
    H=initial_state,
    setpoints=setpoints,
    weights=weights,
    exponents=exponents,
    effects=effects,
    palatability_bonus=palatability_bonus,
    normal_food_reward=normal_food_reward,
)

### Configuration de l'agent

In [40]:
# Initialisation de la Q-table
state_size = 21  # Taille pour inclure les états de 0 à 20
action_size = 2  # Deux actions possibles : nourriture normale ou hyperpalatable
q_table = np.zeros((state_size, action_size))

# Configuration de l'agent Q-learning
alpha = 0.1  # Taux d'apprentissage
gamma = 0.96  # Facteur de discount
epsilon = 0.1  # Facteur d'exploration
agent = QLearning(state_size, action_size, alpha, gamma, epsilon, q_table=q_table)

### Entraînement du modèle

In [41]:
# Hyperparamètres d'apprentissage
episodes = 5000  # Nombre total d'épisodes
reward_history = []  # Historique des récompenses par épisode

# Boucle d'apprentissage
for episode in range(episodes):
    state = env.state.clone().item()  # État initial
    total_reward = 0
    done = False

    while not done:
        # Choisir une action
        action = agent.choose_action(int(state))

        # Effectuer une étape
        next_state, reward, done, _ = env.step(action)

        # Mise à jour Q-learning
        agent.update_q_value(int(state), action, reward, int(next_state.item()))

        # Mise à jour de l'état courant
        state = next_state.item()
        total_reward += reward

    reward_history.append(total_reward)

KeyError: "L'état 10 n'existe pas dans la table Q."

In [ ]:
# Analyse des résultats
plt.plot(reward_history)
plt.title("Historique des récompenses")
plt.xlabel("Épisodes")
plt.ylabel("Récompense cumulée")
plt.show()

# Politique optimale
optimal_policy = [np.argmax(agent.q_table[state]) for state in range(state_size)]
print("Politique optimale :", optimal_policy)